# Lesson 17: Linear Regression

Welcome to Lesson 17! Throughout the course you will complete assignments like this one. You can't learn technical subjects without hands-on practice, so these assignments are an important part of the course.

Collaborating on labs is more than okay -- it's encouraged! You should rarely remain stuck for more than a few minutes on a question, so ask a post to the discussion board or ask your instructor for help. Explaining things is beneficial, too -- the best way to solidify your knowledge of a subject is to explain it. You should **not** just copy/paste someone else's code, but rather work together to gain understanding of the task you need to complete. 

To receive credit for this assignment, answer all questions correctly and submit before the deadline.

**Due Date:** 

**Collaboration Policy:** Data science is a collaborative activity. While you may talk with others about the labs, we ask that you **write your solutions individually**. If you do discuss the assignments with others **please include their names below** (it's a good way to learn your classmates' names).

**Collaborators:** 

List collaborators here.

## Today's Lesson

In today's lab, you'll learn about:

- linear regression.

Let's get started!

## Words of Caution

Remember to run the cell below. It's for setting up the environment so you can have access to what's needed for this lesson. For now, don't worry about what it means: we'll learn more about what's inside of it in the next few lessons.

In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

**Question 1.** In the cell below there are some functions for plotting. You don't have to understand how any of the functions in the cell work, since they use things we have let to learn. 

In [ ]:
def resize_window(lim=3.5):
    plt.xlim(-lim, lim)
    plt.ylim(-lim, lim)
    
def draw_line(slope=0, intercept=0, x=make_array(-4, 4), color='r'):
    y = x*slope+intercept
    plt.plot(x, y, color=color)
    
def draw_vertical_line(x_position, color='black'):
    x = make_array(x_position, x_position)
    y = make_array(-4, 4)
    plt.plot(x, y, color=color)
    
def make_correlated_data(r):
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x+(np.sqrt(1-r**2))*z
    return x, y

def r_table(r):
    """
    Generate a table of 1000 x,y data points in standard units
    whose correlation is approximately equal to r
    """
    np.random.seed(8)
    x, y = make_correlated_data(r)
    return Table().with_columns('x', x, 'y', y)

## Prediction lines

In [ ]:
example = r_table(0.99)
example.show(3)

In [ ]:
example.scatter('x', 'y')

In [ ]:
def nn_prediction_example(x_val, how_close=0.25):
    """Predicts y-value for x based on the example table, 
       using points within the specified number of standard units.
    """
    neighbors = example.where(
        'x', 
        are.between(x_val-how_close, x_val+how_close)
    )
    return np.mean(neighbors.column('y'))    

In [ ]:
nn_prediction_example(-0.5)

In [ ]:
example = example.with_columns(
    'Predicted y', 
    example.apply(nn_prediction_example, 'x'))

In [ ]:
example.scatter('x')

In [ ]:
example.scatter('x')
draw_line(slope=1, color='red')
resize_window()

In [ ]:
example = r_table(0)
example.scatter('x', 'y')
draw_line(slope=0.5, color='red')
resize_window()

In [ ]:
example = example.with_columns(
    'Predicted y', 
    example.apply(nn_prediction_example, 'x'))

In [ ]:
example = example.with_column(
    'Predicted y', example.apply(nn_prediction_example, 'x'))

Table.interactive_plots()
example.scatter('x')
draw_line(slope=0.5, color='red')
resize_window()

In [ ]:
example = r_table(0.5)
example.scatter('x', 'y')
resize_window()

In [ ]:
example = r_table(0.5)

example = example.with_column(
    'Predicted y', example.apply(nn_prediction_example, 'x'))
example.scatter('x', 'y')
draw_vertical_line(1.5, color='blue')
draw_line(slope=1, intercept=0, color='red')
resize_window()

In [ ]:
example = example.with_column('Predicted y', example.apply(nn_prediction_example, 'x'))
example.scatter('x')
draw_vertical_line(1.5, color='blue')
draw_line(slope=1, color='red')
resize_window()

In [ ]:
example.scatter('x')
draw_line(slope=1, intercept=0, color='red')
draw_line(slope=0.5, intercept=0, color='dodgerblue')
resize_window()

## Linear Regression: Defining the Line

In [ ]:
def standard_units(x):
    """Converts an array x to standard units.
    """
    return (x-np.mean(x))/np.std(x)

def correlation(t, x, y):
    x_su = standard_units(t.column(x))
    y_su = standard_units(t.column(y))
    return np.mean(x_su*y_su)

**Question 2.** Finish the code to find the slope and intercept of the regression line.

**Hint:** See the lesson for how to define these.

In [ ]:
def slope(t, x, y):
    ...

def intercept(t, x, y):
    ...

In [ ]:
example = r_table(0.5)
slope(example, 'x', 'y')

## Galton height data

In [ ]:
galton = Table.read_table('data/galton.csv')

heights = Table().with_columns(
    'MidParent', galton.column('midparentHeight'),
    'Child', galton.column('childHeight'))
heights

In [ ]:
m = galton.where('gender', 'female')
np.mean(m.column('childHeight'))

In [ ]:
np.mean(m.column('mother'))

In [ ]:
def nn_prediction_galton(h):
    """Return a prediction of the height of a child 
    whose parents have a midparent height of h.
    
    The prediction is the average height of the children 
    whose midparent height is in the range h plus or minus 0.5 inches.
    """
    neighbors = heights.where(
        'MidParent', are.between(h - 0.5, h + 0.5))
    return np.mean(neighbors.column('Child'))

In [ ]:
heights_with_predictions = heights.with_column(
    'Average neighbor prediction', 
    heights.apply(nn_prediction_galton, 'MidParent'))

**Question 3.** Make sure you define the functions slope and intercept before moving on.

In [ ]:
galton_slope = slope(heights, 'MidParent', 'Child')
galton_intercept = intercept(heights, 'MidParent', 'Child')
galton_slope, galton_intercept

In [ ]:
heights.take(123)

Let's make a prediction for the height of these parents' children.


In [ ]:
heights_with_predictions.where('MidParent', are.equal_to(69.48))

In [ ]:
heights_with_predictions = heights_with_predictions.with_column(
    'Regression Prediction', 
    galton_slope*heights.column('MidParent') + galton_intercept
)
heights_with_predictions

In [ ]:
heights_with_predictions.scatter('MidParent')